In [1]:
from auxillary import adjustPrices, rocApproximation
from indicators import createFeatures
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm, lognorm
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pandas as pd

In [2]:
symbol = "MSFT"
ticker = yf.Ticker(symbol).history("max")
df = adjustPrices(ticker)
cont_vars, cat_vars = createFeatures(df)


In [3]:
scaler = StandardScaler()
scaled_cont_vars = scaler.fit_transform(cont_vars)
pd.DataFrame(scaled_cont_vars, columns= cont_vars.columns)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ADX,Aroon_Up,Aroon_Down,CCI,Disparity,KST,MACD,OBV,RSI,RVI,Stochastic_Oscillator
0,-0.695659,-0.695695,-0.695525,-0.695435,25.461921,-0.060953,-0.030865,NaN,NaN,NaN,NaN,NaN,NaN,-0.131318,-3.634787,NaN,NaN,NaN
1,-0.695457,-0.695675,-0.695320,-0.695354,6.542103,-0.060953,-0.030865,NaN,NaN,NaN,NaN,NaN,NaN,-0.131005,-3.520276,2.806345,NaN,NaN
2,-0.695376,-0.695655,-0.695238,-0.695314,1.966889,-0.060953,-0.030865,NaN,NaN,NaN,NaN,NaN,NaN,-0.130607,-3.470790,2.806345,NaN,NaN
3,-0.695335,-0.695655,-0.695279,-0.695375,0.256832,-0.060953,-0.030865,NaN,NaN,NaN,NaN,NaN,NaN,-0.130538,-3.495972,2.088653,NaN,NaN
4,-0.695396,-0.695715,-0.695320,-0.695415,-0.262737,-0.060953,-0.030865,NaN,NaN,NaN,NaN,NaN,NaN,-0.130649,-3.513769,1.638426,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9338,3.138392,3.172824,3.170758,3.198990,-0.658952,-0.060953,-0.030865,0.224300,1.223649,-0.705917,5.862796,0.974946,0.795727,4.787473,0.826580,1.361875,-0.084620,1.353631
9339,3.176113,3.159443,3.183744,3.184528,-0.864379,-0.060953,-0.030865,0.370296,1.111834,-0.819444,4.734732,0.703339,0.864944,4.967511,0.817334,1.160132,-0.305060,1.224509
9340,3.185713,3.188615,3.207255,3.183852,-0.839776,-0.060953,-0.030865,0.507281,1.223649,-0.932971,4.530278,0.556407,0.903936,5.047849,0.807737,1.149637,-0.183895,1.049963
9341,3.167055,3.144455,3.169665,3.145466,-0.938071,-0.060953,-0.030865,0.648120,1.111834,-1.046497,2.285998,0.251939,0.928428,4.903443,0.799538,0.550675,0.101261,0.479436


In [9]:
for i in scaled_cont_vars.columns:
    col_name = i + "_ROC"
    scaled_cont_vars[col_name] = rocApproximation(scaled_cont_vars[i])

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
# Log normal percentiles
log_norm_cols = ["ADX", "ADX_ROC", "Aroon_Up_Roc", "Aroon_Down_ROC"]
for i in log_norm_cols:
    p = lognorm(loc = scaled_cont_vars[i].mean(), scale = scaled_cont_vars[i].std()).cdf(scaled_cont_vars[i])
    col_name = i + "_Percentile"
    scaled_cont_vars[col_name] = p

In [ ]:
# Normal percentiles
norm_cols = ["CCI", "Disparity", "KST", "MACD", "OBV", "RSI", "RVI", "Stochastic_Oscillator_ROC", "RVI_ROC", "KST_ROC", "MACD_ROC", "OBV_ROC", "RSI_ROC"]

for i in norm_cols:
    p = norm(loc = scaled_cont_vars[i].mean(), scale = scaled_cont_vars[i].std()).cdf(scaled_cont_vars[i])
    col_name = i + "_Percentile"
    scaled_cont_vars[col_name] = p

In [ ]:
regular_df = pd.concat([scaled_cont_vars, cat_vars])


In [ ]:

pca = PCA(n_components=len(regular_df.columns)//2)

#fit PCA model to data
pca_fit = pca.fit(regular_df)

variance_explained = pca.explained_variance_ratio_


In [ ]:
k = []
silhouettes = []
wcv = []
for i in range(2, len(regular_df.columns) + 1):
    kmeans = KMeans(n_clusters=i, n_init="auto").fit(regular_df.dropna())
    wcv.append(kmeans.inertia_)
    k.append(i)
    silhouettes.append(silhouette_score(regular_df.dropna(), kmeans.labels_))

plt.plot(k, wcv, "o-")